In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict

In [2]:
torch.set_grad_enabled(False)

In [3]:
def get_x():
    x_pt = torch.rand((1,3,8,224,224)) * 255
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

In [4]:
x_tf, x_pt = get_x()

## Convert Weights

Initiate model and Load PyTorch Weights

In [5]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

In [6]:
pt_model = SwinTransformer3D_pt(**cfg,drop_rate=0.4, drop_path_rate=0., isTest= True)
tf_model = SwinTransformer3D(**cfg,drop_rate=0.4, drop_path_rate=0., isTest= True)
x_tf, x_pt = get_x()



basic_pt, z= pt_model(x_pt)
print("-------")

basic_tf, y = tf_model(x_tf)
checkpoint = torch.load(f'{name}.pth')



new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 
pt_model.eval()

c:\Python\Python396\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


get_window_size parameters (4, 56, 56) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 56, 56, (4, 7, 7), (0, 3, 3))
attn_mask (64, 196, 196)
get_window_size parameters (4, 56, 56) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 56, 56, (4, 7, 7), (0, 3, 3))
attn_mask (64, 196, 196)
get_window_size parameters (4, 28, 28) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 28, 28, (4, 7, 7), (0, 3, 3))
attn_mask (16, 196, 196)
get_window_size parameters (4, 28, 28) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 28, 28, (4, 7, 7), (0, 3, 3))
attn_mask (16, 196, 196)
get_window_size parameters (4, 14, 14) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 14, 14, (4, 7, 7), (0, 3, 3))
attn_mask (4, 196, 196)
get_window_size parameters (4, 14, 14) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
compute mask parameters (4, 14, 14, (4, 7, 7), (0, 3, 3))
attn_mask (4, 196, 196)
get_window_size parameters (4,

Convert Functions

In [7]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


### convert

In [8]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [9]:
x_tf, x_pt = get_x()

In [24]:
layers_output_tf, y = tf_model(x_tf)
print("------")
layers_output_pt, z= pt_model(x_pt)

y.shape, z.shape


self.compute_mask_info {'shape_of_input': (2, 96, 2, 56, 56), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)}
get_window_size parameters (4, 56, 56) (8, 7, 7) (0, 0, 0)
(4, 7, 7) (0, 0, 0)
get_window_size parameters (4, 56, 56) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)

self.compute_mask_info {'shape_of_input': (2, 192, 2, 28, 28), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)}
get_window_size parameters (4, 28, 28) (8, 7, 7) (0, 0, 0)
(4, 7, 7) (0, 0, 0)
get_window_size parameters (4, 28, 28) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)

self.compute_mask_info {'shape_of_input': (2, 384, 2, 14, 14), 'window_size': (8, 7, 7), 'shift_size': (4, 3, 3)}
get_window_size parameters (4, 14, 14) (8, 7, 7) (0, 0, 0)
(4, 7, 7) (0, 0, 0)
get_window_size parameters (4, 14, 14) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
get_window_size parameters (4, 14, 14) (8, 7, 7) (0, 0, 0)
(4, 7, 7) (0, 0, 0)
get_window_size parameters (4, 14, 14) (8, 7, 7) (4, 3, 3)
(4, 7, 7) (0, 3, 3)
get_window_size paramete

(TensorShape([1, 768, 4, 7, 7]), torch.Size([1, 768, 4, 7, 7]))

In [29]:
# Eplore the output of all layers
for layer in layers_output_pt:
    print("--------------",layer, "-------------\n TF: ", layers_output_tf[layer].numpy()[:1,:1,:1,:1,:10], "\n PT: ", layers_output_pt[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n" )

-------------- PatchEmbed -------------
 TF:  [[[[[-1.52765    -1.8028401  -1.3727719  -1.512826    0.75105476
     -0.6499636  -0.21560025 -1.6660732  -0.58690006 -0.07118201]]]]] 
 PT:  [[[[[-1.52765    -1.8028399  -1.372772   -1.5128261   0.751055
     -0.64996356 -0.21560037 -1.6660732  -0.58690006 -0.07118207]]]]] 

-------------- drop_out -------------
 TF:  [[[[[-1.52765    -1.8028401  -1.3727719  -1.512826    0.75105476
     -0.6499636  -0.21560025 -1.6660732  -0.58690006 -0.07118201]]]]] 
 PT:  [[[[[-1.52765    -1.8028399  -1.372772   -1.5128261   0.751055
     -0.64996356 -0.21560037 -1.6660732  -0.58690006 -0.07118207]]]]] 

-------------- basic layer1 -------------
 TF:  [[[[[ 0.6290703   0.48750663 -0.14188242  0.41177928  0.28953955
      0.8019364   0.7212719   0.549551    0.40311396  0.27723506]]]]] 
 PT:  [[[[[ 0.6290704   0.48750666 -0.1418823   0.41177952  0.2895401
      0.8019364   0.7212718   0.5495511   0.40311405  0.277235  ]]]]] 

-------------- basic layer2 --

In [30]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:
    print("Testing", layer)
    np.testing.assert_allclose(layers_output_tf[layer].numpy(), layers_output_pt[layer].detach().numpy(), 1e-4, 1e-4)

Testing PatchEmbed
Testing drop_out
Testing basic layer1
Testing basic layer2
Testing basic layer3


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 1 / 150528 (0.000664%)
Max absolute difference: 0.00127792
Max relative difference: 0.3161512
 x: array([[[[[ 2.521386e-01,  1.541693e-01,  3.045982e-01, ...,
            1.599499e-01,  2.934550e-01,  3.617281e-01],
          [ 1.779881e-01,  1.574906e-01,  3.287339e-01, ...,...
 y: array([[[[[ 2.521383e-01,  1.541691e-01,  3.045980e-01, ...,
            1.599497e-01,  2.934552e-01,  3.617281e-01],
          [ 1.779877e-01,  1.574907e-01,  3.287338e-01, ...,...

In [40]:
e = 1e-4
np.testing.assert_allclose(layers_output_tf["basic layer4"].numpy(), layers_output_pt["basic layer4"].detach().numpy(), e, e)


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 149169 / 150528 (99.1%)
Max absolute difference: 3.9597473
Max relative difference: 1190.8073
 x: array([[[[[ 1.013054e+00,  6.505743e-01,  1.003217e+00, ...,
            8.176944e-01, -5.685449e-01, -5.212045e-01],
          [ 2.717099e-01,  2.975306e-01, -3.709410e-01, ...,...
 y: array([[[[[ 9.997696e-01,  6.757068e-01,  1.006461e+00, ...,
            7.999148e-01, -5.728335e-01, -5.302565e-01],
          [ 2.555471e-01,  2.936881e-01, -3.771242e-01, ...,...

In [41]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-2, 1e-2)


AssertionError: 
Not equal to tolerance rtol=0.01, atol=0.01

Mismatched elements: 17569 / 150528 (11.7%)
Max absolute difference: 0.24690652
Max relative difference: 1237.8418
 x: array([[[[[ 2.691030e-01,  2.987706e-01,  3.199229e-01, ...,
            2.782986e-01, -8.607621e-02, -7.679231e-02],
          [ 7.984339e-02,  1.169502e-01, -6.092245e-02, ...,...
 y: array([[[[[ 2.675702e-01,  3.133111e-01,  3.174837e-01, ...,
            2.696784e-01, -8.652331e-02, -7.905345e-02],
          [ 7.668141e-02,  1.140875e-01, -6.171951e-02, ...,...

In [ ]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:
    print("Testing", layer)
    np.testing.assert_allclose(layers_output_tf[layer].numpy(), layers_output_pt[layer].detach().numpy(), 1e-4, 1e-4)

### PT basic layer outputs comparison

In [ ]:
attempts = 10
# x_pt = torch.rand((1,3,8,224,224))

outputs = []

for i in range(attempts):
    layer_out , result = pt_model(x_pt)
    outputs.append(layer_out)

i = 0
for layer in outputs[0]:
    print("--------------", layer, "---------------")
    for idx, layer_out in enumerate(outputs) :
        print(f"attempt {idx} : ",layer_out[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n")
    print()


### All close testing

Compare the first attempt with another attempt. Enter the attempt value in the following cell

In [ ]:
attempt_no = 2        # Change the attempt_no value to compare the first attempt with another attempt


In [ ]:
# PatchEmbed Layer
output1 = outputs[0]["PatchEmbed"]
output2 =  outputs[attempt_no]["PatchEmbed"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer1
output1 = outputs[0]["basic layer1"]
output2 =  outputs[attempt_no]["basic layer1"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer2

output1 = outputs[0]["basic layer2"]
output2 =  outputs[attempt_no]["basic layer2"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer3

output1 = outputs[0]["basic layer3"]
output2 =  outputs[attempt_no]["basic layer3"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer4

output1 = outputs[0]["basic layer4"]
output2 =  outputs[attempt_no]["basic layer4"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Final Output

output1 = outputs[0]["Final Output"]
output2 =  outputs[attempt_no]["Final Output"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)